#Cleaning Images

##Import Packages

In [ ]:
import warnings
warnings.filterwarnings("ignore")

from IPython.display import display, HTML, IFrame
import math
import ast
from PIL import Image
import numpy as np
import pandas as pd
from shapely.geometry import MultiPoint
from scipy import stats
import matplotlib.pyplot as plt
import bs4, re
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from urllib.request import urlopen
import requests
import json
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
#function for getting polygon from the google map overlay javascript file (and also locating the .js file for that)

def getpoly(url):
  #url would be the url of the neighborhood's/city's page
  data = urlopen(url)
  source = data.read().decode('utf-8')

  #regex to search
  regex = r"https:\/\/pp\.walk\.sc\/_\/s\/_p\/hood\/[a-z0-9]+\.js"

  matches = re.finditer(regex, source, re.MULTILINE)

  for matchNum, match in enumerate(matches, start=1):
      jslink = match.group()

  #jslink is the javascript page link for the google map overlay
  data = urlopen(jslink)
  jspage = data.read().decode('utf-8')

  regex = r"(?<=window\.activeEntity = reconstituteEntity)\(.*\)"

  matches = re.finditer(regex, jspage, re.MULTILINE)

  for matchNum, match in enumerate(matches, start=1):
   polycode = match.group()

  #get overlay dictionary
  dic = json.loads(polycode[1:-1])

  #get encoded path string from dictionary
  path = dic['path']

  return path

In [ ]:
def polydecode(en):
  eval_res, tempfile = js2py.run_file("googlepolydecoder.js")
  return tempfile.decodePolyline(en).to_list()

In [ ]:
i = 0

while i < mydata.shape[0]:
  a = str(getpoly(mydata['URL'][i]))
  mydata['EnPath'][i] = a
  mydata['Polygon'][i] = polydecode(a)
  i = i+1

In [ ]:
mydata.head(3)

,Rank#,Name,Walk Score,Transit Score,Bike Score,Population,URL,Polygon,EnPath
0,1,Downtown,98,100,70,"5,059",https://www.walkscore.com/WA/Seattle/Downtown,"[{'latitude': 47.61096, 'longitude': -122.3404...",o_raHxruiV}CuEgEgLgTcr@n[hJvFn@`DDxDs@nDcBlO}L...
1,2,International District,98,100,83,"3,396",https://www.walkscore.com/WA/Seattle/Internati...,"[{'latitude': 47.59888, 'longitude': -122.3290...",_toaHjksiVaAMsNF@yYdIaLpD_EEen@`T}NBdoASfUkBxEcNA
2,3,Belltown,98,97,76,"14,163",https://www.walkscore.com/WA/Seattle/Belltown,"[{'latitude': 47.61847, 'longitude': -122.3284...",mnsaH`hsiVnMtBfTbr@fEfL|CtEqHhOlOdTVi@t@dAv@oA...


In [ ]:
def mapPara(txt,typ):
  dictionary = {"{":'(', "}":')', " ":'', "[":'', "]":''}
  pol = txt
  pol = pol.replace("'latitude'",'lat')
  pol = pol.replace("'longitude'",'lng')

  transTable = txt.maketrans(dictionary)
  txt = txt.translate(transTable)
  txt = txt.replace("'latitude':",'')
  txt = txt.replace("'longitude':",'')

  lst = list(ast.literal_eval(txt))
  points = MultiPoint(lst)
  b = points.bounds
  #n = b[2] - 0.25*(b[2]-b[0])
  #s = b[0] + 0.25*(b[2]-b[0])
  #e = b[3] - 0.25*(b[3]-b[1])
  #w = b[1] + 0.25*(b[3]-b[1])
  x = 0.5*(b[0]+b[2])
  y = 0.5*(b[1]+b[3])

  if typ == 'bounds':
    return '{north:' + str(b[2]) + ', south:' + str(b[0]) + ', west:' + str(b[1]) + ', east:' + str(b[3]) + ',}'
    #return '{north:' + str(n) + ', south:' + str(s) + ', west:' + str(w) + ', east:' + str(e) + ',}'
  elif typ == 'center':
    return '{lat:' + str(x) + ', lng:' + str(y)+ '}'
  elif typ == 'polygon':
    return pol
  else:
    print ("Invalid Argument: Secondary Argument Must Be 'bounds','polygon' or 'center'")

In [ ]:
def genHTML(p):
  url = 'https://kirthi-b.github.io/ML_Final/map.html'

  req = requests.get(url, 'html.parser')
  html = req.text

  html = html.replace("{p:}",str(mapPara(p,'polygon')))
  html = html.replace("{b:}",str(mapPara(p,'bounds')))
  html = html.replace("{c:}",str(mapPara(p,'center')))

  return html

In [ ]:
i = 0
path_half = '/content/gdrive/MyDrive/Columbia/2 Spring 22/ML for Urban Data/ML Final/Project Files/Urban-Mobility-Index/Data/Training/NYC/Maps/'

while i < mydata.shape[0]:
  url = path_half + 'HTML/' + str(mydata['Name'][i]) + '.html'
  with open(url, 'w') as f:
    f.write(genHTML(str(mydata['Polygon'][i])))
  i = i+1

In [ ]:
i = 0

while i < mydata.shape[0]:
  url = path_half + 'HTML/' + str(mydata['Name'][i]) + '.html'
  dir = path_half + 'IMG/'
  convertapi.convert('png', {
      'File': url,
      'ConversionDelay': '4'
  }, from_format = 'html').save_files(dir)
  i = i+1

KeyboardInterrupt: ignored